In [ ]:
import os
import json
import codecs
import pandas as pd
from pandas.io.json import json_normalize
from json.decoder import JSONDecodeError

def json2excel(fileName, sheetName, columnName, visitorDataFrameColumns,
               visitorDataFrameColumnsList, path2folder, 
               startDate, endDate, outputFolder):
""""   
Function json2excel takes an exported json file containing data of a Google Analytics User ID as an input 
and transform it into an Excel file.
(See example of the function call in the 2nd cell below).
"""" 
"""" 
Arguments: 
fileName - the name of a file containing the table of the summary data of User IDs.
an exported json file containing data of a Google Analytics User ID.
sheetName - the name of the sheet in a file containing the table of the summary data of User IDs.
columnName - the name of the column in a file containing the table of the summary data of User IDs.
visitorDataFrameColumns - an argument to the nested function json2dataFrame.
visitorDataFrameColumnsList - an argument to the nested function json2dataFrame.
path2folder - path to the folder that contains the exported json file.
startDate - start date of the Date Range (type string).
endDate - end date of the Date Range (type string).
outputFolder - path the folder to output the Excel file that contains data of a Google Analytics User ID.

Returns:
flag - the state of function json2excel performance ('OK').
outputFileName - the name of the the Excel file that contains data of a Google Analytics User ID.
logFileName - the name of a log file that contains results of function json2excel performance.
"""" 

    # receiving visitors GA IDs (aka Client ID) into a list from excel file
    clientsID = pd.read_excel(io = path2folder + '/' + fileName, 
                              sheet_name = sheetName,
                              usecols = 0, dtype = {columnName : str})

    clientsIDlist = clientsID[columnName].values.tolist()

    # constructng a list of json file names with visitors data
    clientDataFileList = clientsID[columnName].values.tolist()

    for i in range(0, len(clientDataFileList)):
        clientDataFileList[i] = path2folder + '/' + clientDataFileList[i] + '.json'

    # getting names of columns for DataFrame visitorsData from the 1st json file
    with open(clientDataFileList[0], encoding='utf8') as f:
        rawData = json.load(f)

    # getting visitor data from the 1st json file into DataFrame visitorsData
    visitorsData = json2dataFrame(clientsIDlist[0], 
                                  rawData, 
                                  'dates', 
                                  ['sessions', 'activities', 'details'], 
                                  visitorDataFrameColumns, 
                                  visitorDataFrameColumnsList)
    logFileName = '{5}{6}{0}{2}{3}{2}{4}{1}'.format('visitorsData_log', '.txt', '_',
                                              startDate, endDate, outputFolder, '/')
    logFile = open(logFileName, 'w')
    logFile.writelines('Parameters:\n')
    logFile.writelines('{0}{1}{2}'.format('Start date = ', startDate, '\n'))
    logFile.writelines('{0}{1}{2}'.format('End date = ', endDate, '\n'))
    logFile.writelines('\n')
    logFile.writelines('Starting processing:\n')                                        
    logFile.writelines('{0}{2}{1}{3}'.format(0,clientsID.iat[0,0],
                                             '. ID = ', ' - OK.\n'))

    # getting visitor data from json files (starting from the 2nd) into -->
    # --> DataFrame visitorDataFrame -->
    # --> and combining them into single dataFrame visitorsData
    counter1 = []
    counter2 = []
    #NEW CODE
    counter3 = []

    for j in range(1, len(clientsIDlist)):
    #for j in range(1, 20):
        if os.stat(clientDataFileList[j]) == 0:
            logFile.writelines('{0}{2}{1}{4}'.format(j,
                                                     clientsID.iat[j,0],
                                                     '. ID = ',
                                                     ' - Empty json file!\n'))
            counter1.append(j)

        else:
            try:
                with open(clientDataFileList[j], encoding='utf8') as f:
                    rawData = json.load(f)
                    #NEW CODE
                    if rawData.get('dates') and len(rawData['dates']) != 0:
                        logFile.writelines('{0}{2}{1}{3}'.format(j,
                                                                 clientsID.iat[j,0],
                                                                 '. ID = ',
                                                                 ' - OK.\n'))
                        visitorDataFrame = json2dataFrame(clientsIDlist[j],
                                                         rawData, 
                                                         'dates', 
                                                         ['sessions', 'activities', 'details'], 
                                                         visitorDataFrameColumns,
                                                         visitorDataFrameColumnsList)
                        visitorsData = pd.concat([visitorsData,visitorDataFrame],
                                                 ignore_index = True)
                    else:
                        logFile.writelines('{0}{1}{2}{3}'.format(j,
                                                                 '. ID = ',
                                                                 clientsID.iat[j,0],
                                                                 ' - json file holds no data!\n'))
                        counter3.append(j) 

            except JSONDecodeError:
                logFile.writelines('{0}{1}{2}{3}'.format(j,
                                                         '. ID = ',
                                                         clientsID.iat[j,0],
                                                         ' - json file is incorrectly encoded!\n'))
                counter2.append(j) 


    # saving DataFrame visitorsData in an excel file
    
    outputFileName = '{5}{6}{0}{2}{3}{2}{4}{1}'.format('visitorsData', '.xlsx', '_',
                                                 startDate, endDate, outputFolder,'/')
    
    visitorsData.to_excel(outputFileName)
    logFile.writelines('\n')
    logFile.writelines('{0}{1}{2}'.format('1. ', len(clientsID),
                                          ' json files in the folder. \n'))
    #MODIFIED CODE
    logFile.writelines('{0}{1}{2}'.format('2. ',
                                          len(clientsID)-len(counter1)-len(counter2)-len(counter3),
                                          ' files successfully processed.\n'))
    
    logFile.writelines('{0}{1}{2}'.format('3. ', len(counter1), ' file(s) empty\n'))
    for k in range(len(counter1)):
        logFile.writelines('{0}{1}{2}{3}{4}'.format('   ', k+1, '). ',
                                                    clientsID.iat[counter1[k],0],
                                                    '.json.\n'))
    logFile.writelines('{0}{1}{2}'.format('4. ', len(counter2), ' file(s) incorrectly coded\n'))
    for k in range(len(counter2)):
        logFile.writelines('{0}{1}{2}{3}{4}'.format('   ', k+1, '). ', 
                                                    clientsID.iat[counter2[k],0],
                                                    '.json.\n'))
    #NEW CODE
    logFile.writelines('{0}{1}{2}'.format('5. ', len(counter3), 
                                          ' file(s) have no data\n'))
    for k in range(len(counter3)):
        logFile.writelines('{0}{1}{2}{3}{4}'.format('   ', k+1, '). ', 
                                                    clientsID.iat[counter3[k],0],
                                                    '.json.\n'))

    logFile.close()
    flag = 'OK'
    return flag, outputFileName, logFileName


def json2dataFrame(ID, input_json, node, path, columnsNames, columnsList):

"""" 
Function json2dataFrame uses an imported function json_normalize to unroll json data into a custom Pandas DataFrame.
"""" 
""""
Arguments:
ID - ID of UserID in an json file.
input_json - json data.
node - a node for the imported function json_normalize.
path - a list of upper level columns' names.
columnsNames - a list of columns's named nested related to the list of upper level columns' names (the argument path above).
columnsList - a list of columns's named nested related to columnsNames (the argument path above).

Returns: 
output_dataset  - a Pandas DataFrame containg data of the exported json file with data of a Google Analytics User ID.
"""" 

#    print(ID)
# parsing json into dataFrame output_dataset
    output_dataset = json_normalize(data = input_json[node], 
                                    record_path = path,
                                    meta = columnsNames, errors = 'ignore')

    output_dataset_columns = output_dataset.columns.values

    columns2delete = []
    for item in output_dataset_columns:
        if item in columnsList:
            None
        else:
            columns2delete.append(item)    
#    columns2delete.remove('URL страницы')
#    columns2delete.remove('Заголовок страницы')
    if columns2delete != []:
        output_dataset.drop(columns2delete, axis = 1, inplace = True)
    else: None

    output_dataset['Client ID'] = ID

    for i in range(output_dataset.shape[0]):
        output_dataset.iat[i,5] = (output_dataset.iat[i,5])['PAGEVIEW']

    for j in range(len(output_dataset)):
        if output_dataset.iat[j,10] == 'PAGEVIEW':
            output_dataset.iat[j,0] = output_dataset.iat[j,0][0]
            output_dataset.iat[j,1] = output_dataset.iat[j,1][0]
        else: None


    output_dataset.rename({output_dataset.columns.values[5] : output_dataset.columns.values[5]+' - PAGEVIEW'},
                           axis = 'columns', inplace = True)

    output_dataset.rename({output_dataset.columns.values[0] : 'sessions.activities.details - '+output_dataset.columns.values[0]},
                           axis = 'columns', inplace = True)
            
    output_dataset.rename({output_dataset.columns.values[1] : 'sessions.activities.details - '+output_dataset.columns.values[1]},
                           axis = 'columns', inplace = True)

    return output_dataset    


In [ ]:
import os
import datetime

fileName = ''
path2folder = ''

startDate = '' #'dd.mm.yyyy'
endDate = '' #'dd.mm.yyyy'

outputFolder = ''

sheetName = ''
columnName = ''

visitorDataFrameColumns = ['date', 'hasGoal', 'sessionCount',
                           ['sessions', 'activitySummary'],
                           ['sessions', 'channel'],
                           ['sessions', 'deviceCategory'],
                           ['sessions', 'duration'],
                           ['sessions', 'activities', 'time'],
                           ['sessions', 'activities', 'type']]
visitorDataFrameColumnsList = ['URL страницы', 'Заголовок страницы',
                               'date', 'hasGoal', 'hasRevenue', 'sessionCount',
                               'sessions.activitySummary', 'sessions.channel',
                               'sessions.deviceCategory','sessions.duration',
                               'sessions.activities.time',
                               'sessions.activities.type']      

print(datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
print('\n')

result = json2excel(fileName, sheetName, columnName, visitorDataFrameColumns,
                    visitorDataFrameColumnsList, path2folder, 
                    startDate, endDate, outputFolder)
print(result)
print('\n')
print(datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))